In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader


In [10]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data.iloc[idx, 1:].values.astype(np.uint8).reshape((28, 28,1))
        label = self.data.iloc[idx, 0]
        if self.transform:
            image = self.transform(image)
        return image, label

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = CustomDataset(csv_file='mnist_train.csv', transform=transform)
test_dataset = CustomDataset(csv_file='mnist_test.csv', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [12]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 10)
        
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 1024)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN()

In [13]:
def train(model, train_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")

In [14]:
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy


In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
train(model, train_loader, criterion, optimizer, epochs=10)


Epoch 1/10, Loss: 2.3089
Epoch 2/10, Loss: 2.3088
Epoch 3/10, Loss: 2.3089
Epoch 4/10, Loss: 2.3089
Epoch 5/10, Loss: 2.3087
Epoch 6/10, Loss: 2.3087
Epoch 7/10, Loss: 2.3085
Epoch 8/10, Loss: 2.3082
Epoch 9/10, Loss: 2.3090
Epoch 10/10, Loss: 2.3087


In [17]:
def cw_l1_attack(model, x, y, targeted=False, c=0.1, kappa=0, lr=1, steps=1000, eps=0.01):
    # Implementation of CW L1 attack
    x_adv = x.clone().detach().requires_grad_(True)
    for i in range(steps):
        output = model(x_adv)
        loss = nn.CrossEntropyLoss()(output, y)

        if targeted:
            target_labels = torch.argsort(output, dim=1)[:, -1]
            correct = target_labels == y
            loss += (1 - kappa) * nn.CrossEntropyLoss()(output, target_labels[correct])

        else:
            correct = torch.argmax(output, dim=1) == y
            loss -= (1 - kappa) * nn.CrossEntropyLoss()(output, target_labels[~correct])

        optimizer = optim.Adam([x_adv], lr=lr)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        diff = x_adv - x
        diff = torch.clamp(diff, -eps, eps)
        x_adv = torch.clamp(x + diff, 0, 1).detach().requires_grad_(True)

    return x_adv

# Implement CW L2 Attack
def cw_l2_attack(model, x, y, targeted=False, c=0.1, kappa=10, lr=1, steps=1000, eps=0.01):
    # Implementation of CW L2 attack
    x_adv = x.clone().detach().requires_grad_(True)
    for i in range(steps):
        output = model(x_adv)
        loss = nn.CrossEntropyLoss()(output, y)

        if targeted:
            target_labels = torch.argsort(output, dim=1)[:, -1]
            correct = target_labels == y
            loss += (1 - kappa) * nn.CrossEntropyLoss()(output, target_labels[correct])

        else:
            correct = torch.argmax(output, dim=1) == y
            loss -= (1 - kappa) * nn.CrossEntropyLoss()(output, target_labels[~correct])

        optimizer = optim.Adam([x_adv], lr=lr)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        diff = x_adv - x
        diff = eps * diff / torch.norm(diff, p=2)
        x_adv = torch.clamp(x + diff, 0, 1).detach().requires_grad_(True)

    return x_adv


In [18]:
accuracy_cw_l1 = evaluate(model, test_loader)
print(f'Accuracy after CW L1 Attack: {accuracy_cw_l1:.2f}%')

# Evaluate the accuracy after CW L2 Attack
accuracy_cw_l2 = evaluate(model, test_loader)
print(f'Accuracy after CW L2 Attack: {accuracy_cw_l2:.2f}%')

Accuracy after CW L1 Attack: 10.10%
Accuracy after CW L2 Attack: 10.10%


In [19]:
import torch.nn as nn

# Define the model architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(12*12*64, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2, 2)
        x = x.view(-1, 12*12*64)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define create_ensemble function
def create_ensemble(num_models):
    ensemble = []
    for _ in range(num_models):
        model = CNN()
        ensemble.append(model)# Adversarial training with CW L1 attack
def adversarial_training_cw_l1(model, train_loader, epochs, epsilon=0.01):
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            inputs_adv = cw_l1_attack(model, inputs, labels, eps=epsilon)
            optimizer.zero_grad()
            outputs = model(inputs_adv)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

# Adversarial training with CW L2 attack
def adversarial_training_cw_l2(model, train_loader, epochs, epsilon=0.01):
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            inputs_adv = cw_l2_attack(model, inputs, labels, eps=epsilon)
            optimizer.zero_grad()
            outputs = model(inputs_adv)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

# Defensive distillation
def distillation_training(ensemble, teacher_model, train_loader, epochs, temperature=3):
    for model in ensemble:
        model.train()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.KLDivLoss(reduction='batchmean')
        for epoch in range(epochs):
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs_teacher = teacher_model(inputs) / temperature
                outputs_student = model(inputs) / temperature
                loss = criterion(torch.log_softmax(outputs_student, dim=1), 
                                 torch.softmax(outputs_teacher, dim=1))
                loss.backward()
                optimizer.step()

# Create an ensemble of models
num_models = 3
ensemble = create_ensemble(num_models)

# Train teacher model for defensive distillation
teacher_model = CNN().to(device)
teacher_model.train()
optimizer_teacher = optim.Adam(teacher_model.parameters(), lr=0.001)
criterion_teacher = nn.CrossEntropyLoss()
for epoch in range(epochs):
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer_teacher.zero_grad()
        outputs = teacher_model(inputs)
        loss = criterion_teacher(outputs, labels)
        loss.backward()
        optimizer_teacher.step()

# Adversarial training with CW L1 attack for each model in the ensemble
for model in ensemble:
    adversarial_training_cw_l1(model, train_loader, epochs)

# Adversarial training with CW L2 attack for each model in the ensemble
for model in ensemble:
    adversarial_training_cw_l2(model, train_loader, epochs)

# Defensive distillation for each model in the ensemble
distillation_training(ensemble, teacher_model, train_loader, epochs)

# Evaluate the models
# Evaluate individual models
for i, model in enumerate(ensemble):
    acc = evaluate(model)
    print(f"Model {i+1} Test Accuracy: {acc:.2f}%")

# Evaluate ensemble
ensemble_accuracy = evaluate(ensemble[0])  # Evaluate using the first model
print(f"Ensemble Test Accuracy: {ensemble_accuracy:.2f}%")

    return ensemble



IndentationError: unexpected indent (707744501.py, line 113)

In [22]:
import torch.nn as nn

# Define the model architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(5*5*64, 128) 
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2, 2)
        x = x.view(-1, 5*5*64) 
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    
def cw_l1_attack(model, x, y, targeted=False, c=0.1, kappa=0, lr=1, steps=1000, eps=0.01):
    # Implementation of CW L1 attack
    x_adv = x.clone().detach().requires_grad_(True)
    for i in range(steps):
        output = model(x_adv)
        loss = nn.CrossEntropyLoss()(output, y)

        if targeted:
            target_labels = torch.argsort(output, dim=1)[:, -1]
            correct = target_labels == y
            loss += (1 - kappa) * nn.CrossEntropyLoss()(output, target_labels[correct])

        else:
            target_labels = torch.argsort(output, dim=1)[:, -1]
            correct = torch.argmax(output, dim=1) == y
            loss -= (1 - kappa) * nn.CrossEntropyLoss()(output, target_labels[~correct])

        optimizer = optim.Adam([x_adv], lr=lr)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        diff = x_adv - x
        diff = torch.clamp(diff, -eps, eps)
        x_adv = torch.clamp(x + diff, 0, 1).detach().requires_grad_(True)

    return x_adv

# Define create_ensemble function
def create_ensemble(num_models):
    ensemble = []
    for _ in range(num_models):
        model = CNN()
        ensemble.append(model)
    return ensemble

# Adversarial training with CW L1 attack
def adversarial_training_cw_l1(model, train_loader, epochs, epsilon=0.01):
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            inputs_adv = cw_l1_attack(model, inputs, labels, eps=epsilon)
            optimizer.zero_grad()
            outputs = model(inputs_adv)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

# Adversarial training with CW L2 attack
def adversarial_training_cw_l2(model, train_loader, epochs, epsilon=0.01):
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            inputs_adv = cw_l2_attack(model, inputs, labels, eps=epsilon)
            optimizer.zero_grad()
            outputs = model(inputs_adv)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

# Defensive distillation
def distillation_training(ensemble, teacher_model, train_loader, epochs, temperature=3):
    for model in ensemble:
        model.train()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.KLDivLoss(reduction='batchmean')
        for epoch in range(epochs):
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs_teacher = teacher_model(inputs) / temperature
                outputs_student = model(inputs) / temperature
                loss = criterion(torch.log_softmax(outputs_student, dim=1), 
                                 torch.softmax(outputs_teacher, dim=1))
                loss.backward()
                optimizer.step()

# Create an ensemble of models
num_models = 3
ensemble = create_ensemble(num_models)

# Train teacher model for defensive distillation
teacher_model = CNN().to(device)
teacher_model.train()
optimizer_teacher = optim.Adam(teacher_model.parameters(), lr=0.001)
criterion_teacher = nn.CrossEntropyLoss()
for epoch in range(epochs):
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer_teacher.zero_grad()
        outputs = teacher_model(inputs)
        loss = criterion_teacher(outputs, labels)
        loss.backward()
        optimizer_teacher.step()

# Adversarial training with CW L1 attack for each model in the ensemble
for model in ensemble:
    adversarial_training_cw_l1(model, train_loader, epochs)

# Adversarial training with CW L2 attack for each model in the ensemble
for model in ensemble:
    adversarial_training_cw_l2(model, train_loader, epochs)

# Defensive distillation for each model in the ensemble
distillation_training(ensemble, teacher_model, train_loader, epochs)

# Evaluate the models
# Evaluate individual models
for i, model in enumerate(ensemble):
    acc = evaluate(model)
    print(f"Model {i+1} Test Accuracy: {acc:.2f}%")

# Evaluate ensemble
ensemble_accuracy = evaluate(ensemble[0])  # Evaluate using the first model
print(f"Ensemble Test Accuracy: {ensemble_accuracy:.2f}%")


KeyboardInterrupt: 